In [ ]:
# 1. 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 2. Ultralytics(YOLOv8) 설치 및 필요 라이브러리 임포트
!pip install ultralytics -q
!pip install opencv-python-headless -q

import os
import cv2
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from ultralytics import YOLO

# GPU 확인
print(f"Setup Complete. Using GPU: {torch.cuda.get_device_name(0)}")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 76.4 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Setup Complete. Using GPU: NVIDIA A100-SXM4-80GB


In [ ]:
import os
import glob
from tqdm import tqdm

# 1. 경로 설정 (드라이브 경로)
base_dir = '/content/drive/MyDrive/cv2/Laboro_Tomato'
target_dirs = [
    os.path.join(base_dir, 'train/labels'),
    os.path.join(base_dir, 'val/labels')  # 범인은 보통 여기에 숨어있습니다
]

# 2. 변환 로직
def fix_label(cls_id):
    cls_id = int(cls_id)
    if cls_id == 3: return 0 # 작은 완숙 -> 완숙
    if cls_id == 4: return 1 # 작은 반숙 -> 반숙
    if cls_id == 5: return 2 # 작은 초록 -> 초록
    return cls_id

print("🕵️‍♂️ 잘못된 라벨(3,4,5) 색출 및 수정 작업을 시작합니다...")

fixed_files = 0
total_errors = 0

for dir_path in target_dirs:
    if not os.path.exists(dir_path):
        print(f"⚠️ 경로 없음: {dir_path}")
        continue

    txt_files = glob.glob(os.path.join(dir_path, "*.txt"))
    print(f"📂 {dir_path} 검사 중... ({len(txt_files)}개 파일)")

    for txt_path in tqdm(txt_files):
        with open(txt_path, 'r') as f:
            lines = f.readlines()

        new_lines = []
        is_modified = False

        for line in lines:
            parts = line.strip().split()
            if not parts: continue

            cls_id = int(parts[0])

            # 3 이상인 숫자가 발견되면 즉시 수정
            if cls_id >= 3:
                new_id = fix_label(cls_id)
                parts[0] = str(new_id) # 수정된 ID로 교체
                new_lines.append(" ".join(parts) + "\n")
                is_modified = True
                total_errors += 1
            else:
                new_lines.append(line) # 정상이면 그대로 유지

        # 수정된 내용이 있다면 파일 덮어쓰기
        if is_modified:
            with open(txt_path, 'w') as f:
                f.writelines(new_lines)
            fixed_files += 1

print(f"\n✅ 소탕 완료!")
print(f" - 수정된 파일 수: {fixed_files}개")
print(f" - 고쳐진 라벨 수: {total_errors}개")
print("이제 다시 학습을 돌리면 에러가 나지 않을 겁니다.")

🕵️‍♂️ 잘못된 라벨(3,4,5) 색출 및 수정 작업을 시작합니다...
📂 /content/drive/MyDrive/cv2/Laboro_Tomato/train/labels 검사 중... (1286개 파일)


100%|██████████| 1286/1286 [00:06<00:00, 191.36it/s]


📂 /content/drive/MyDrive/cv2/Laboro_Tomato/val/labels 검사 중... (161개 파일)


100%|██████████| 161/161 [00:00<00:00, 388.70it/s]


✅ 소탕 완료!
 - 수정된 파일 수: 0개
 - 고쳐진 라벨 수: 0개
이제 다시 학습을 돌리면 에러가 나지 않을 겁니다.


In [ ]:
# [Cell] 3클래스용 설정 파일(laboro_3class.yaml) 생성
import os

# 파일 내용 작성 (경로와 클래스 정보 수정)
yaml_content = """
path: /content/drive/MyDrive/cv2/Laboro_Tomato
train: train/images
val: val/images

# 클래스 수 변경 (6 -> 3)
nc: 3

# 이름 변경 (크기 구분 없이 숙성도만)
names:
  0: fully_ripened
  1: half_ripened
  2: green
"""

# 저장 경로
yaml_path = '/content/drive/MyDrive/cv2/Laboro_Tomato/laboro_3class.yaml'

# 파일 쓰기
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print(f"🎯 설정 파일 생성 완료: {yaml_path}")
print("이제 모델이 '크기'는 무시하고 '숙성도'만 봅니다.")

🎯 설정 파일 생성 완료: /content/drive/MyDrive/cv2/Laboro_Tomato/laboro_3class.yaml
이제 모델이 '크기'는 무시하고 '숙성도'만 봅니다.


In [ ]:
import shutil
import os

base_dir = '/content/drive/MyDrive/cv2/Laboro_Tomato'
train_label_dir = os.path.join(base_dir, 'train/labels')
val_label_dir = os.path.join(base_dir, 'val/labels')

# 폴더가 있다면 삭제하고 다시 깨끗하게 생성
if os.path.exists(train_label_dir):
    shutil.rmtree(train_label_dir)
os.makedirs(train_label_dir, exist_ok=True)

if os.path.exists(val_label_dir):
    shutil.rmtree(val_label_dir)
os.makedirs(val_label_dir, exist_ok=True)

print("✅ 기존 라벨 폴더를 초기화했습니다. 이제 깨끗합니다.")

✅ 기존 라벨 폴더를 초기화했습니다. 이제 깨끗합니다.


In [ ]:
import json
import numpy as np
from tqdm import tqdm

CLASSES = [
    'b_fully_ripened', 'b_half_ripened', 'b_green',
    'l_fully_ripened', 'l_half_ripened', 'l_green'
]

def convert_coco_to_yolo_seg_clean(json_file, save_dir):
    with open(json_file, 'r') as f:
        data = json.load(f)

    images = {item['id']: item for item in data['images']}
    cat_map = {}
    for cat in data['categories']:
        if cat['name'] in CLASSES:
            cat_map[cat['id']] = CLASSES.index(cat['name'])

    for ann in tqdm(data['annotations'], desc=f"Converting {os.path.basename(json_file)}"):
        image_id = ann['image_id']
        if image_id not in images: continue

        img_info = images[image_id]
        img_w = img_info['width']
        img_h = img_info['height']

        file_name = os.path.splitext(img_info['file_name'])[0]
        txt_path = os.path.join(save_dir, file_name + '.txt')

        class_id = cat_map.get(ann['category_id'])
        if class_id is None: continue

        # --- 핵심: Segmentation 데이터가 있는 경우만 처리 ---
        if 'segmentation' in ann and len(ann['segmentation']) > 0:
            for seg in ann['segmentation']:
                poly = np.array(seg).reshape(-1, 2)
                poly[:, 0] /= img_w
                poly[:, 1] /= img_h
                poly = poly.flatten().tolist()

                # 점이 3개 미만(데이터 길이 < 6)인 잘못된 폴리곤은 패스
                if len(poly) < 6: continue

                poly_str = ' '.join([f"{x:.6f}" for x in poly])

                with open(txt_path, 'a') as f:
                    f.write(f"{class_id} {poly_str}\n")

        # 주의: else 문을 제거하여 Box 좌표가 아예 생성되지 않도록 함
        # (YOLO-seg 학습에는 Box 좌표만 있는 라인은 필요 없음)

# 변환 실행
json_path_train = os.path.join(base_dir, 'annotations/train.json')
json_path_test = os.path.join(base_dir, 'annotations/test.json')

print("Train 라벨 재생성 중...")
convert_coco_to_yolo_seg_clean(json_path_train, train_label_dir)

print("Val 라벨 재생성 중...")
if os.path.exists(json_path_test):
    convert_coco_to_yolo_seg_clean(json_path_test, val_label_dir)

print("✅ 라벨 재생성 완료 (중복 및 Box 좌표 제거됨)")

Train 라벨 재생성 중...


Converting train.json: 100%|██████████| 7781/7781 [00:21<00:00, 354.52it/s]


Val 라벨 재생성 중...


Converting test.json: 100%|██████████| 1996/1996 [00:06<00:00, 307.71it/s]

✅ 라벨 재생성 완료 (중복 및 Box 좌표 제거됨)


In [ ]:
import os
import glob

# 1. 누락된 경로 변수 선언 (스크린샷 기반 절대 경로)
# 주의: Colab 마운트 경로(/content/drive/...)가 포함되어야 함
base_dir = '/content/drive/MyDrive/cv2/Laboro_Tomato'
train_label_dir = os.path.join(base_dir, 'train', 'labels')

# 2. 경로 유효성 방어 로직 (경로가 틀리면 여기서 바로 확인 가능)
if not os.path.exists(train_label_dir):
    print(f"[오류] 경로를 찾을 수 없습니다: {train_label_dir}")
    print("구글 드라이브 마운트 상태나 폴더명을 다시 확인하십시오.")
else:
    print(f"[확인] 타겟 경로: {train_label_dir}")

    # 3. 기존 로직 실행
    sample_files = glob.glob(os.path.join(train_label_dir, '*.txt'))

    if sample_files:
        target_file = sample_files[0]
        with open(target_file, 'r') as f:
            print(f"파일명: {os.path.basename(target_file)}")
            print("내용 (첫 2줄):")
            lines = f.readlines()
            for line in lines[:2]:
                print(line.strip()[:100] + "...")
    else:
        print("생성된 라벨 파일(.txt)이 없습니다. 데이터셋 생성 단계를 점검하십시오.")

[확인] 타겟 경로: /content/drive/MyDrive/cv2/Laboro_Tomato/train/labels
파일명: IMG_1090.txt
내용 (첫 2줄):
2 0.652970 0.322040 0.680650 0.331160 0.701580 0.346350 0.715090 0.368630 0.706990 0.394960 0.695510...
2 0.584780 0.336220 0.604360 0.314950 0.608410 0.293180 0.606390 0.276980 0.590860 0.256220 0.566550...


In [ ]:
from ultralytics import YOLO
import torch

# GPU 확인
print(f"🔥 사용 중인 GPU: {torch.cuda.get_device_name(0)}")

# 모델 로드
model = YOLO('yolov8m-seg.pt')

# 학습 시작
results = model.train(
    data='/content/drive/MyDrive/cv2/Laboro_Tomato/laboro_3class.yaml',
    epochs=100,
    patience=20,
    batch=32,
    imgsz=1024,
    device=0,
    workers=16,
    project='/content/drive/MyDrive/cv2/runs',
    name='tomato_3class_REAL',
    exist_ok=True,
    cache=True,

    # 아까 설정하신 옵션들
    optimizer='SGD',
    lr0=0.001,
    cos_lr=True,
    mixup=0.1,
    copy_paste=0.1,
    mosaic=1.0,
)

🔥 사용 중인 GPU: NVIDIA A100-SXM4-80GB
Ultralytics 8.4.6 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/cv2/Laboro_Tomato/laboro_3class.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolov8m-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=tomato_3class_REAL, nbs=64, nms=False, opset=None, 